In [ ]:
# Example of pKa

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys
import copy

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage, MolToImage

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)
show_progress = True

## Define a molecule you like

In [ ]:
smiles = "CCCCCCO"  # 
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

## Get some 3D conformers (RDKit)

In [ ]:
molobj = ppqm.tasks.generate_conformers(molobj, n_conformers=1)

In [ ]:
ppqm.jupyter.show_molobj(molobj)

In [ ]:
torsion_indices = [1, 2, 3 ,4]

In [ ]:
MolToImage(
   molobj,
   highlightAtoms=torsion_indices,
   size=(500,500),
)

In [ ]:
conformer = molobj.GetConformer()
origin = conformer.GetPositions()

In [ ]:
angle = Chem.rdMolTransforms.GetDihedralDeg(conformer, *indices)
angle

In [ ]:
steps = 100
delta_angles = np.linspace(0, 360, steps)
delta_angles = delta_angles[1:]

In [ ]:
for step in delta_angles:
    
    conformer_prime = rdkit.Chem.Conformer(conformer)
    
    angle_prime = angle + step
    
    Chem.rdMolTransforms.SetDihedralDeg(conformer_prime, *torsion_indices, angle_prime)
    molobj.AddConformer(conformer_prime, assignId=True)
    

In [ ]:
molobj.GetNumConformers()

In [ ]:
ppqm.jupyter.show_molobj(molobj, align_conformers=False)

## Calculate the energy barrier

In [ ]:
xtb = ppqm.XtbCalculator(
    scr="_tmp_directory_",
    n_cores=2,
    cmd="xtb",
    show_progress=show_progress,
)

def calculate_energies(molobj):
    
    xtb_options = {
        "gfn": 2
    }

    results = xtb.calculate(molobj, xtb_options)
    energies = [result["scc_energy"] for result in results]
    
    energies = np.array(energies)
    energies *= ppqm.units.hartree_to_kcalmol
    energies -= np.min(energies)
    
    return energies

In [ ]:
energies = calculate_energies(molobj)

## Show the energy barrier

In [ ]:
_ = plt.plot([angle] + list(angle+delta_angles), energies)